In [1]:
#  This tutorial show that only pandas is used
import numpy as np
import os
from tqdm.notebook import tqdm

In [2]:
# Path to the directory with all components
base_dir = '../../WoD/training'
height = 64
width = 2650
w_cut=[490,2170] # 1680
h_cut=[0,64] # 64
in_dir = f'{base_dir}/preprocess'
out_dir = f'{base_dir}/preprocess_mini'
os.makedirs(out_dir, exist_ok=True)
# search all .bin files in the in_dir directory
input_files = sorted([f for f in os.listdir(in_dir) if f.endswith('.bin')])

# go through the list of files
for file in tqdm(input_files, desc="Processing file"):
    data = np.fromfile(f'{in_dir}/{file}', dtype=np.float32)
    data = data.reshape((height, width, 10))  # reshape to (64, 2650, 10)
    # cut the data
    data = data[h_cut[0]:h_cut[1], w_cut[0]:w_cut[1], :]
    data.tofile(f'{out_dir}/{file}')



Processing file:   0%|          | 0/23691 [00:00<?, ?it/s]